# Agent Testing Notebook

This notebook demonstrates the modernized Agent with SOUL.md identity and Memory module.

## Setup

In [1]:
import sys
from pathlib import Path

# Add src to path if needed
sys.path.insert(0, str(Path.cwd()))

from src.agent import Agent
from src.config import Config

## Initialize Agent

The agent will automatically load its identity from `SOUL.md`.

In [2]:
config = Config(
    llama_cpp_url="http://localhost:8080",
    use_chat_api=True,
    chat_template="chatml",
    max_iterations=5,
    temperature=0.7,
    max_tokens=2048,
)

agent = Agent(
    config=config,
    db_path="notebook_test_sessions.db",
    lazy_rag=True,
)

print(f"✅ Agent initialized")
print(f"📊 Tools loaded: {len(agent.tools.list_tools())}")
print(f"🧠 Memory module: {type(agent.memory).__name__}")

✅ Agent initialized
📊 Tools loaded: 30
🧠 Memory module: Memory


## Check SOUL

Let's verify the agent loaded its identity from SOUL.md:

In [3]:
print("System Prompt (first 500 chars):")
print("=" * 60)
print(agent.system_prompt[:500])
print("...")

System Prompt (first 500 chars):
# SOUL of the Agent

## Core Identity
You are a specialized AI agent for time series analysis and forecasting. You are helpful, precise, and focused.

## Critical Rules
1. **ALWAYS stay on topic** - Answer the user's actual question directly
2. **NEVER generate irrelevant content** - If you don't know something, say so
3. **Be concise** - Avoid rambling or repetitive text
4. **Use tools when appropriate** - You have powerful analysis tools available

## Prime Directives
1. **Be Helpful**: Direct
...


## Test Basic Chat

In [ ]:
response = agent.chat(
    "Hello! Can you introduce yourself?",
    session_id="test_session_2",
    verbose=True
)

print("\n" + "=" * 60)
print("Agent Response:")
print("=" * 60)
print(response)

/data/back_home/baseline/foreblocks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 2561.47it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-base-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



--- Iter 1 • session=test_ses ---

[context→llm (iter 1)]
01. system
# SOUL of the Agent

## Core Identity
You are a specialized AI agent for time series analysis and forecasting. You are helpful, precise, and focused.

## Critical Rules
1. **ALWAYS stay on topic** - Answer the user's actual question directly
2. **NEVER generate irrelevant content** - If you don't know something, say so
3. **Be concise** - Avoid rambling or repetitive text
4. **Use tools when appropriate** - You have powerful analysis tools available

## Prime Directives
1. **Be Helpful**: Direct

[... truncated 11971 chars ...]
02. user
Hello! Can you introduce yourself?
03. assistant
are more than 200 miles (320 km) from the nearest coast. The only exception is the Merauke region in the southern part of Papua, which is a little over 200 miles from the nearest coast. Merauke, located in the southern part of Papua, is the only place in the region that is more than 200 miles from the nearest coast. This distance is sig

## Test Tool Usage

Let's test if the agent can use tools from the skills directory:

In [5]:
# List available tools
print("Available Tools:")
print("=" * 60)
for tool_name in sorted(agent.tools.registry.keys())[:10]:  # Show first 10
    tool = agent.tools.registry[tool_name]
    print(f"• {tool_name}: {tool.description[:60]}...")

Available Tools:
• acf_pacf_peaks: Top ACF/PACF lags by magnitude....
• change_points: Detect structural breaks using ruptures....
• comprehensive_analysis: Full analysis pipeline with tool-to-tool calling....
• compute_multivariate_stats: Correlations and per-series stats for multivariate data....
• compute_statistics: Compute descriptive statistics....
• create_sample_data: Create synthetic time series for testing....
• cross_validate: Rolling-origin cross-validation using Nixtla's built-in `.cr...
• detect_anomalies: Detect anomalies - returns counts only (indices cached)....
• detect_trend: Detect overall trend direction....
• detrend: Remove polynomial trend via fitting....


In [6]:
# Test a tool-using query
response = agent.chat(
    "Can you check what data analysis tools you have available?",
    session_id="test_session_1",
    verbose=True
)

print("\n" + "=" * 60)
print("Agent Response:")
print("=" * 60)
print(response)


--- Iter 1 • session=test_ses ---

[context→llm (iter 1)]
01. system
# SOUL of the Agent

## Core Identity
You are a specialized AI agent for time series analysis and forecasting. You are helpful, precise, and focused.

## Critical Rules
1. **ALWAYS stay on topic** - Answer the user's actual question directly
2. **NEVER generate irrelevant content** - If you don't know something, say so
3. **Be concise** - Avoid rambling or repetitive text
4. **Use tools when appropriate** - You have powerful analysis tools available

## Prime Directives
1. **Be Helpful**: Direct

[... truncated 11971 chars ...]
02. user
Hello! Can you introduce yourself?
03. assistant
are more than 200 miles (320 km) from the nearest coast. The only exception is the Merauke region in the southern part of Papua, which is a little over 200 miles from the nearest coast. Merauke, located in the southern part of Papua, is the only place in the region that is more than 200 miles from the nearest coast. This distance is sig

## Test Memory Module

The Memory module handles conversation history and RAG context:

In [7]:
# Check session history
sessions = agent.memory.list_sessions()
print(f"Active sessions: {len(sessions)}")
for sid, timestamp in sessions[:5]:  # Show first 5
    print(f"  • {sid[:16]}... @ {timestamp}")

Active sessions: 1
  • test_session_1... @ 2026-02-16 13:35:31


In [8]:
# Test conversation continuity
response = agent.chat(
    "What did I just ask you about?",
    session_id="test_session_1",
    verbose=False
)

print("Agent remembers:")
print(response)

Agent remembers:
are more notable than the 'Cocktail' (1988) and 'The Muppets' (1979). The 'Cocktail' soundtrack, released by Atlantic Records, features a mix of 1980s pop and rock music, including hits like "Wild Wild West" by Will Smith, "Dancing Machine" by The Jackson 5, and "The First Time Ever I Saw Your Face" by Roberta Flack. This soundtrack is particularly notable for its iconic status and the way it captured the essence of the 1980s. In contrast, the soundtrack for 'The Muppets' (1979) is a unique blend of music and humor, reflecting the whimsical and imaginative nature of the Muppets. The film's soundtrack includes performances by the Muppets themselves, as well as contributions from notable artists like Frank Oz and Jim Henson. The 1979 release, produced by Jim Henson and Frank Oz, is considered a classic in its own right, with a mix of original songs and popular tunes that have become beloved by generations. Another notable 1979 release is "The Muppets Go to the Movies, " 

## Test Direct Tool Execution

You can also call tools directly without LLM involvement:

In [9]:
# Example: if you have a specific tool loaded
# Uncomment and modify based on your available tools

# result = agent.run_tool_direct(
#     tool_name="get_data_info",
#     arguments={},
#     persist_to_history=False
# )
# print(result)

print("Direct tool execution available via agent.run_tool_direct()")

Direct tool execution available via agent.run_tool_direct()


## Cleanup

In [10]:
# Reset session if needed
agent.reset("test_session_1")
print("✅ Session reset")

✅ Session reset


## Summary

This notebook demonstrated:
- ✅ Agent initialization with SOUL.md identity
- ✅ Memory module integration
- ✅ Basic chat functionality
- ✅ Tool discovery and usage
- ✅ Conversation history management
- ✅ Direct tool execution

The modernized agent architecture provides:
- **Cleaner separation** between identity (SOUL.md) and code
- **Modular memory** management via the Memory class
- **Interactive REPL** mode for command-line usage
- **Flexible tool system** with skills-based loading